In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, KFold, LeaveOneOut
from sklearn.preprocessing import Imputer
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import category_encoders as ce
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from catboost import CatBoostClassifier

In [3]:
train_A = pd.read_csv('data/A_hhold_train.csv', index_col='id')
train_B = pd.read_csv('data/B_hhold_train.csv', index_col='id')
train_C = pd.read_csv('data/C_hhold_train.csv', index_col='id')

In [4]:
test_A = pd.read_csv('data/A_hhold_test.csv', index_col='id')
test_B = pd.read_csv('data/B_hhold_test.csv', index_col='id')
test_C = pd.read_csv('data/C_hhold_test.csv', index_col='id')

In [5]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

In [6]:
#funcetion used to centrailize the numeric data 
def standardize(df, numeric_only=True): 
    
    """
    :type df: pd.DataFrame
    :type numeric_only: bool
    
    :rtype df: pd.DataFrame
    """
    
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df

In [7]:
#function used to preprocess the data(simply drop the poor and country columns)
def preporcess(original_train, original_test, op='one_hot'):
    
    """
    :type original_train: pd.DataFrame
    :type original_test: pd.DataFrame
    :type op: string
    
    :rtype x_train: pd.DataFrame
    :rtype y_train: pd.DataFrame
    :rtype x_test: pd.DataFrame
    """
    
    #Get the copy of orginal data
    x_train = original_train.copy()
    x_test = original_test.copy()

    #Get the label of the train set
    y_train = pd.DataFrame(x_train['poor'])
    
    #drop the 'poor' and 'country' columns
    x_train = x_train.drop(['poor', 'country'], axis=1)
    x_test = x_test.drop(['country'], axis=1)
    
    #get the list of coulumns that need to be encoded
    columns = list(x_train.select_dtypes(include=['object']).columns)
    
    #encode the features according to the 'op' operations
    if op == 'one_hot':
        encoder = ce.one_hot.OneHotEncoder(cols=columns)
    elif op == 'binary':
        encoder = ce.binary.BinaryEncoder(cols=columns)
    elif op == 'ordinal':
        encoder = ce.ordinal.OrdinalEncoder(cols=columns)
    elif op == 'backward':
        encoder = ce.backward_difference.BackwardDifferenceEncoder(cols=columns)
    elif op == 'polynomial':
        encoder = ce.polynomial.PolynomialEncoder(cols=columns)
    elif op == 'sum':
        encoder = ce.sum_coding.SumEncoder(cols=columns)
    elif op == 'hashing':
        encoder = ce.hashing.HashingEncoder(cols=columns)
    elif op == 'helmert':
        encoder = ce.helmert.HelmertEncoder(cols=columns)
    
    #fit the encoder and transform the datasets
    encoder.fit(x_train)
    x_train = encoder.transform(x_train)
    x_test = encoder.transform(x_test)
    
    #for columns with Nan, plug in with median value
    imputed_train = x_train.copy()
    imputed_test = x_test.copy()
    
    # make new columns indicating what will be imputed
    cols_with_missing = (col for col in x_train.columns 
                                 if x_train[col].isnull().any())
    
    #insert new colums and tell which row is missing 
    for col in cols_with_missing:
        imputed_train[col + '_was_missing'] = imputed_train[col].isnull()
        imputed_test[col + '_was_missing'] = imputed_test[col].isnull()
    
    imp = Imputer(missing_values='NaN', strategy='median',axis=0)
    x_train = imp.fit_transform(imputed_train)
    x_test = imp.transform(imputed_test)
       
    return x_train, y_train.values.ravel(), x_test

In [8]:
x_train_A, y_train_A, x_test_A = preporcess(original_train=train_A, original_test=test_A, op='one_hot')
x_train_B, y_train_B, x_test_B = preporcess(original_train=train_B, original_test=test_B, op='one_hot')
x_train_C, y_train_C, x_test_C = preporcess(original_train=train_C, original_test=test_C, op='one_hot')

In [9]:
def cross_validate(model, X, y, cv=5, oversample=False, early_stop=False):
    
    seed = randint(1,9999)
    result_log_loss = np.zeros(cv)
    result_roc_auc = np.zeros(cv)
    
    kf = StratifiedKFold(n_splits=cv)
    
    i = 0
    
    for train_index, dev_index in kf.split(X,y):
        x_train = X[train_index]
        y_train = y[train_index]
        x_dev = X[dev_index]
        y_dev = y[dev_index]
        
        if oversample == True:
            x_train, y_train = RandomOverSampler().fit_sample(x_train, y_train)
            
        if early_stop == True:
            model.fit(X=x_train, y=y_train, eval_set=[(x_train,y_train),(x_dev,y_dev)], eval_metric='logloss', early_stopping_rounds=20)
        else:
            model.fit(x_train, y_train)
        
        pred_proba = model.predict_proba(x_dev)[:,-1]
        
        result_log_loss[i] = log_loss(y_pred=pred_proba, y_true=y_dev)
        result_roc_auc[i] = roc_auc_score(y_score=pred_proba, y_true=y_dev)
        
        i += 1
        
    return result_log_loss.mean(), result_roc_auc.mean()

In [10]:
def gridSearch(model, grid_params, X, y, oversample=False, early_stop=False, cv=5):
    
    grids = ParameterGrid(grid_params)
    result_score = np.zeros([len(grids), 2])
    
    i = 0
    
    for grid in grids:
        model.set_params(**grid)
        logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
        result_score[i][0] = logScore
        result_score[i][1] = aucScore
        i += 1
    
    return grids, result_score

In [11]:
def model_feature_evaluate(feature_model, predict_model, X, y, oversample=False, early_stop=False, cv=5):
    feature_model.fit(X,y)
    feature_model = SelectFromModel(feature_model, prefit=True)
    X_new = feature_model.transform(X)
    logScore, aucScore = cross_validate(predict_model, X_new, y, cv, oversample, early_stop)
    return logScore, aucScore

def base_model_evaluate(model, X, y, oversample=False, early_stop=False, cv=5):
    logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
    return logScore, aucScore

def feature_transform(feature_model, X_train, Y_train, X_test):
    feature_model.fit(X_train, Y_train)
    model = SelectFromModel(feature_model, prefit=True)
    X_train_new = model.transform(X_train)
    X_test_new = model.transform(X_test)
    return X_train_new, X_test_new

In [12]:
x_train_new_A, x_test_new_A = feature_transform(XGBClassifier(), x_train_A, y_train_A, x_test_A)
x_train_new_B, x_test_new_B = feature_transform(XGBClassifier(), x_train_B, y_train_B, x_test_B)
x_train_new_C, x_test_new_C = feature_transform(XGBClassifier(), x_train_C, y_train_C, x_test_C)

# Country A Model Parameters

In [28]:
#xgb data
#lr
lr_params_A = {}
lr_params_A['C'] = 1
lr_params_A['penalty'] = 'l1'

#xgb
xgb_params_A = {}
xgb_params_A['learning_rate'] = 0.05
xgb_params_A['n_estimators'] = 800
xgb_params_A['max_depth'] = 3
xgb_params_A['colsample_bytree'] = 0.9


#lgb
lgb_params_A = {}
lgb_params_A['learning_rate'] = 0.05
lgb_params_A['num_iterations'] = 800
lgb_params_A['max_depth'] = 3
lgb_params_A['colsample_bytree'] = 0.6
lgb_params_A['min_child_weight'] = 4

# Country B Model Parameters

In [33]:
#xgb data
#lr
lr_params_B = {}
lr_params_B['C'] = 0.5
lr_params_B['penalty'] = 'l2'

#lgb
lgb_params_B = {}
lgb_params_B['learning_rate'] = 0.01
lgb_params_B['max_depth'] = 8
lgb_params_B['num_iterations'] = 1000
lgb_params_B['colsample_bytree'] = 0.4
lgb_params_B['subsample'] = 0.7

#xgb
xgb_params_B = {}
xgb_params_B['learning_rate'] = 0.05
xgb_params_B['n_estimators'] = 500
xgb_params_B['min_child_weight'] = 2
xgb_params_B['colsample_bytree'] = 0.8
xgb_params_B['max_delta_step'] = 3


# Country C Model Parameters

In [37]:
#xgb data
#lr
lr_params_C = {}
lr_params_C['C'] = 1
lr_params_C['penalty'] = 'l1'

#xgb
xgb_params_C = {}
xgb_params_C['learning_rate'] = 0.05
xgb_params_C['n_estimators'] = 300
xgb_params_C['max_depth'] = 3
xgb_params_C['colsample_bytree'] = 0.6
xgb_params_C['max_delta_step'] = 3

#lgb
lgb_params_C = {}
lgb_params_C['learning_rate'] = 0.05
lgb_params_C['num_iterations'] = 150
lgb_params_C['max_depth'] = 4

# Parameters tunning

In [53]:
params = {}
params['weights'] = [[1,1,1,1],[3,1,1,1], [1,3,1,1], [1,1,3,1], [1,1,1,3]]

In [54]:
p, s = gridSearch(model_C, params, x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6206831	total: 15.9ms	remaining: 15.9s
1:	learn: 0.5580744	total: 35.1ms	remaining: 17.5s
2:	learn: 0.5045482	total: 51.7ms	remaining: 17.2s
3:	learn: 0.4537160	total: 71.7ms	remaining: 17.9s
4:	learn: 0.4107796	total: 86.1ms	remaining: 17.1s
5:	learn: 0.3702110	total: 104ms	remaining: 17.2s
6:	learn: 0.3361582	total: 118ms	remaining: 16.7s
7:	learn: 0.3050076	total: 135ms	remaining: 16.8s
8:	learn: 0.2771751	total: 154ms	remaining: 16.9s
9:	learn: 0.2531978	total: 172ms	remaining: 17s
10:	learn: 0.2326573	total: 189ms	remaining: 17s
11:	learn: 0.2122879	total: 204ms	remaining: 16.8s
12:	learn: 0.1954316	total: 233ms	remaining: 17.7s
13:	learn: 0.1811325	total: 251ms	remaining: 17.7s
14:	learn: 0.1668440	total: 271ms	remaining: 17.8s
15:	learn: 0.1549930	total: 287ms	remaining: 17.6s
16:	learn: 0.1438238	total: 303ms	remaining: 17.5s
17:	learn: 0.1358999	total: 363ms	remaining: 19.8s
18:	learn: 0.1271550	total: 432ms	remaining: 22.3s
19:	learn: 0.1141082	total: 478ms	remain

168:	learn: 0.0088168	total: 3.48s	remaining: 17.1s
169:	learn: 0.0087858	total: 3.5s	remaining: 17.1s
170:	learn: 0.0087289	total: 3.52s	remaining: 17.1s
171:	learn: 0.0086656	total: 3.54s	remaining: 17s
172:	learn: 0.0086358	total: 3.56s	remaining: 17s
173:	learn: 0.0085932	total: 3.58s	remaining: 17s
174:	learn: 0.0085221	total: 3.59s	remaining: 16.9s
175:	learn: 0.0084731	total: 3.62s	remaining: 16.9s
176:	learn: 0.0084248	total: 3.68s	remaining: 17.1s
177:	learn: 0.0083663	total: 3.74s	remaining: 17.3s
178:	learn: 0.0083219	total: 3.83s	remaining: 17.6s
179:	learn: 0.0082689	total: 3.86s	remaining: 17.6s
180:	learn: 0.0081931	total: 3.88s	remaining: 17.5s
181:	learn: 0.0081427	total: 3.9s	remaining: 17.6s
182:	learn: 0.0080836	total: 3.93s	remaining: 17.6s
183:	learn: 0.0080574	total: 3.96s	remaining: 17.5s
184:	learn: 0.0080021	total: 3.99s	remaining: 17.6s
185:	learn: 0.0079604	total: 4.01s	remaining: 17.6s
186:	learn: 0.0079195	total: 4.06s	remaining: 17.6s
187:	learn: 0.007866

332:	learn: 0.0037763	total: 7.66s	remaining: 15.3s
333:	learn: 0.0037570	total: 7.68s	remaining: 15.3s
334:	learn: 0.0037384	total: 7.7s	remaining: 15.3s
335:	learn: 0.0037202	total: 7.72s	remaining: 15.3s
336:	learn: 0.0037097	total: 7.75s	remaining: 15.2s
337:	learn: 0.0036929	total: 7.76s	remaining: 15.2s
338:	learn: 0.0036766	total: 7.78s	remaining: 15.2s
339:	learn: 0.0036585	total: 7.83s	remaining: 15.2s
340:	learn: 0.0036359	total: 7.87s	remaining: 15.2s
341:	learn: 0.0036279	total: 7.89s	remaining: 15.2s
342:	learn: 0.0036162	total: 7.91s	remaining: 15.1s
343:	learn: 0.0036114	total: 7.93s	remaining: 15.1s
344:	learn: 0.0035782	total: 7.95s	remaining: 15.1s
345:	learn: 0.0035679	total: 7.97s	remaining: 15.1s
346:	learn: 0.0035585	total: 7.98s	remaining: 15s
347:	learn: 0.0035421	total: 8s	remaining: 15s
348:	learn: 0.0035226	total: 8.02s	remaining: 15s
349:	learn: 0.0035150	total: 8.04s	remaining: 14.9s
350:	learn: 0.0035019	total: 8.06s	remaining: 14.9s
351:	learn: 0.0034940	

495:	learn: 0.0020216	total: 12.5s	remaining: 12.7s
496:	learn: 0.0020131	total: 12.5s	remaining: 12.6s
497:	learn: 0.0020051	total: 12.5s	remaining: 12.6s
498:	learn: 0.0020016	total: 12.5s	remaining: 12.6s
499:	learn: 0.0019936	total: 12.5s	remaining: 12.5s
500:	learn: 0.0019899	total: 12.6s	remaining: 12.5s
501:	learn: 0.0019855	total: 12.6s	remaining: 12.5s
502:	learn: 0.0019814	total: 12.6s	remaining: 12.4s
503:	learn: 0.0019769	total: 12.6s	remaining: 12.4s
504:	learn: 0.0019739	total: 12.6s	remaining: 12.4s
505:	learn: 0.0019686	total: 12.7s	remaining: 12.4s
506:	learn: 0.0019611	total: 12.7s	remaining: 12.3s
507:	learn: 0.0019544	total: 12.7s	remaining: 12.3s
508:	learn: 0.0019526	total: 12.7s	remaining: 12.3s
509:	learn: 0.0019430	total: 12.7s	remaining: 12.2s
510:	learn: 0.0019334	total: 12.8s	remaining: 12.2s
511:	learn: 0.0019307	total: 12.8s	remaining: 12.2s
512:	learn: 0.0019273	total: 12.8s	remaining: 12.2s
513:	learn: 0.0019248	total: 12.8s	remaining: 12.1s
514:	learn: 

...

487:	learn: 0.0019390	total: 12.9s	remaining: 13.5s
488:	learn: 0.0019347	total: 12.9s	remaining: 13.5s
489:	learn: 0.0019304	total: 12.9s	remaining: 13.4s
490:	learn: 0.0019214	total: 12.9s	remaining: 13.4s
491:	learn: 0.0019180	total: 12.9s	remaining: 13.4s
492:	learn: 0.0019110	total: 13s	remaining: 13.3s
493:	learn: 0.0019029	total: 13s	remaining: 13.3s
494:	learn: 0.0018990	total: 13s	remaining: 13.2s
495:	learn: 0.0018962	total: 13s	remaining: 13.2s
496:	learn: 0.0018878	total: 13s	remaining: 13.2s
497:	learn: 0.0018766	total: 13s	remaining: 13.1s
498:	learn: 0.0018746	total: 13.1s	remaining: 13.1s
499:	learn: 0.0018689	total: 13.1s	remaining: 13.1s
500:	learn: 0.0018604	total: 13.1s	remaining: 13s
501:	learn: 0.0018532	total: 13.1s	remaining: 13s
502:	learn: 0.0018479	total: 13.1s	remaining: 13s
503:	learn: 0.0018456	total: 13.2s	remaining: 12.9s
504:	learn: 0.0018422	total: 13.2s	remaining: 12.9s
505:	learn: 0.0018337	total: 13.2s	remaining: 12.9s
506:	learn: 0.0018290	total: 1

649:	learn: 0.0012686	total: 15.8s	remaining: 8.52s
650:	learn: 0.0012645	total: 15.8s	remaining: 8.5s
651:	learn: 0.0012629	total: 15.9s	remaining: 8.47s
652:	learn: 0.0012622	total: 15.9s	remaining: 8.44s
653:	learn: 0.0012597	total: 15.9s	remaining: 8.42s
654:	learn: 0.0012564	total: 15.9s	remaining: 8.39s
655:	learn: 0.0012540	total: 16s	remaining: 8.37s
656:	learn: 0.0012532	total: 16s	remaining: 8.34s
657:	learn: 0.0012513	total: 16s	remaining: 8.31s
658:	learn: 0.0012478	total: 16s	remaining: 8.29s
659:	learn: 0.0012465	total: 16s	remaining: 8.26s
660:	learn: 0.0012454	total: 16.1s	remaining: 8.23s
661:	learn: 0.0012429	total: 16.1s	remaining: 8.2s
662:	learn: 0.0012390	total: 16.1s	remaining: 8.18s
663:	learn: 0.0012370	total: 16.1s	remaining: 8.15s
664:	learn: 0.0012361	total: 16.1s	remaining: 8.12s
665:	learn: 0.0012314	total: 16.1s	remaining: 8.09s
666:	learn: 0.0012305	total: 16.2s	remaining: 8.07s
667:	learn: 0.0012240	total: 16.2s	remaining: 8.04s
668:	learn: 0.0012204	to

809:	learn: 0.0009247	total: 18.8s	remaining: 4.42s
810:	learn: 0.0009221	total: 18.9s	remaining: 4.39s
811:	learn: 0.0009213	total: 18.9s	remaining: 4.37s
812:	learn: 0.0009205	total: 18.9s	remaining: 4.34s
813:	learn: 0.0009184	total: 18.9s	remaining: 4.32s
814:	learn: 0.0009175	total: 18.9s	remaining: 4.29s
815:	learn: 0.0009157	total: 18.9s	remaining: 4.27s
816:	learn: 0.0009140	total: 19s	remaining: 4.25s
817:	learn: 0.0009119	total: 19s	remaining: 4.22s
818:	learn: 0.0009105	total: 19s	remaining: 4.2s
819:	learn: 0.0009099	total: 19s	remaining: 4.17s
820:	learn: 0.0009088	total: 19s	remaining: 4.15s
821:	learn: 0.0009085	total: 19s	remaining: 4.12s
822:	learn: 0.0009068	total: 19.1s	remaining: 4.1s
823:	learn: 0.0009064	total: 19.1s	remaining: 4.08s
824:	learn: 0.0009056	total: 19.1s	remaining: 4.05s
825:	learn: 0.0009050	total: 19.1s	remaining: 4.03s
826:	learn: 0.0009043	total: 19.1s	remaining: 4s
827:	learn: 0.0009031	total: 19.1s	remaining: 3.98s
828:	learn: 0.0009015	total: 

973:	learn: 0.0007269	total: 21.8s	remaining: 582ms
974:	learn: 0.0007265	total: 21.8s	remaining: 559ms
975:	learn: 0.0007251	total: 21.8s	remaining: 537ms
976:	learn: 0.0007247	total: 21.9s	remaining: 514ms
977:	learn: 0.0007243	total: 21.9s	remaining: 492ms
978:	learn: 0.0007239	total: 21.9s	remaining: 470ms
979:	learn: 0.0007234	total: 21.9s	remaining: 447ms
980:	learn: 0.0007224	total: 21.9s	remaining: 425ms
981:	learn: 0.0007220	total: 21.9s	remaining: 402ms
982:	learn: 0.0007213	total: 22s	remaining: 380ms
983:	learn: 0.0007208	total: 22s	remaining: 357ms
984:	learn: 0.0007196	total: 22s	remaining: 335ms
985:	learn: 0.0007179	total: 22s	remaining: 313ms
986:	learn: 0.0007174	total: 22s	remaining: 290ms
987:	learn: 0.0007170	total: 22.1s	remaining: 268ms
988:	learn: 0.0007161	total: 22.1s	remaining: 245ms
989:	learn: 0.0007145	total: 22.1s	remaining: 223ms
990:	learn: 0.0007137	total: 22.1s	remaining: 201ms
991:	learn: 0.0007130	total: 22.1s	remaining: 178ms
992:	learn: 0.0007113	

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [55]:
list(p)

[{'weights': [1, 1, 1, 1]},
 {'weights': [3, 1, 1, 1]},
 {'weights': [1, 3, 1, 1]},
 {'weights': [1, 1, 3, 1]},
 {'weights': [1, 1, 1, 3]}]

In [56]:
s

array([[0.01714127, 0.9995637 ],
       [0.01644251, 0.9995815 ],
       [0.01628891, 0.99959076],
       [0.01692952, 0.99956557],
       [0.02033712, 0.99951045]])

# Model

In [57]:
clf1_A = LGBMClassifier(**lgb_params_A)
clf2_A = XGBClassifier(**xgb_params_A)
clf3_A = CatBoostClassifier(loss_function='Logloss')
clf4_A = LogisticRegression(**lr_params_A)

clf1_B = LGBMClassifier(**lgb_params_B)
clf2_B = XGBClassifier(**xgb_params_B)
clf3_B = CatBoostClassifier(loss_function='Logloss')
clf4_B = LogisticRegression()

clf1_C = LGBMClassifier(**lgb_params_C)
clf2_C = XGBClassifier(**xgb_params_C)
clf3_C = CatBoostClassifier(loss_function='Logloss')
clf4_C = LogisticRegression(**lr_params_C)

model_A = VotingClassifier(estimators=[('lgb', clf1_A), ('xgb', clf2_A), ('cat', clf3_A), ('lr', clf4_A)], voting='soft', weights=[1,1,3,1])
model_B = VotingClassifier(estimators=[('lgb', clf1_B), ('xgb', clf2_B), ('cat', clf3_B), ('lr', clf4_B)], voting='soft')
model_C = VotingClassifier(estimators=[('lgb', clf1_C), ('xgb', clf2_C), ('cat', clf3_C), ('lr', clf4_C)], voting='soft', weights=[1,3,1,1])

model_A.fit(x_train_new_A, y_train_A)
model_B.fit(x_train_new_B, y_train_B)
model_C.fit(x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6749733	total: 37ms	remaining: 37s
1:	learn: 0.6574690	total: 64.7ms	remaining: 32.3s
2:	learn: 0.6406237	total: 93.8ms	remaining: 31.2s
3:	learn: 0.6261026	total: 123ms	remaining: 30.5s
4:	learn: 0.6110564	total: 154ms	remaining: 30.7s
5:	learn: 0.5973595	total: 180ms	remaining: 29.8s
6:	learn: 0.5842524	total: 211ms	remaining: 29.9s
7:	learn: 0.5731438	total: 255ms	remaining: 31.6s
8:	learn: 0.5620291	total: 286ms	remaining: 31.4s
9:	learn: 0.5519159	total: 313ms	remaining: 31s
10:	learn: 0.5417747	total: 344ms	remaining: 30.9s
11:	learn: 0.5327804	total: 429ms	remaining: 35.3s
12:	learn: 0.5245848	total: 511ms	remaining: 38.8s
13:	learn: 0.5165293	total: 556ms	remaining: 39.2s
14:	learn: 0.5087224	total: 585ms	remaining: 38.4s
15:	learn: 0.5008765	total: 611ms	remaining: 37.6s
16:	learn: 0.4948026	total: 642ms	remaining: 37.1s
17:	learn: 0.4886462	total: 671ms	remaining: 36.6s
18:	learn: 0.4816359	total: 700ms	remaining: 36.2s
19:	learn: 0.4755244	total: 735ms	remaining:

166:	learn: 0.2849847	total: 5.27s	remaining: 26.3s
167:	learn: 0.2846283	total: 5.3s	remaining: 26.3s
168:	learn: 0.2842492	total: 5.33s	remaining: 26.2s
169:	learn: 0.2839094	total: 5.36s	remaining: 26.2s
170:	learn: 0.2834614	total: 5.39s	remaining: 26.1s
171:	learn: 0.2829749	total: 5.42s	remaining: 26.1s
172:	learn: 0.2826794	total: 5.45s	remaining: 26.1s
173:	learn: 0.2822703	total: 5.48s	remaining: 26s
174:	learn: 0.2818866	total: 5.51s	remaining: 26s
175:	learn: 0.2815529	total: 5.54s	remaining: 26s
176:	learn: 0.2812489	total: 5.58s	remaining: 25.9s
177:	learn: 0.2807999	total: 5.61s	remaining: 25.9s
178:	learn: 0.2803697	total: 5.63s	remaining: 25.8s
179:	learn: 0.2800029	total: 5.66s	remaining: 25.8s
180:	learn: 0.2797189	total: 5.7s	remaining: 25.8s
181:	learn: 0.2793447	total: 5.73s	remaining: 25.8s
182:	learn: 0.2789012	total: 5.76s	remaining: 25.7s
183:	learn: 0.2785093	total: 5.79s	remaining: 25.7s
184:	learn: 0.2782402	total: 5.82s	remaining: 25.6s
185:	learn: 0.277943

328:	learn: 0.2425974	total: 10.2s	remaining: 20.8s
329:	learn: 0.2424800	total: 10.2s	remaining: 20.7s
330:	learn: 0.2422575	total: 10.2s	remaining: 20.7s
331:	learn: 0.2421116	total: 10.3s	remaining: 20.7s
332:	learn: 0.2418750	total: 10.3s	remaining: 20.6s
333:	learn: 0.2416281	total: 10.3s	remaining: 20.6s
334:	learn: 0.2414245	total: 10.4s	remaining: 20.6s
335:	learn: 0.2412626	total: 10.4s	remaining: 20.6s
336:	learn: 0.2411062	total: 10.4s	remaining: 20.5s
337:	learn: 0.2411048	total: 10.5s	remaining: 20.5s
338:	learn: 0.2408951	total: 10.5s	remaining: 20.4s
339:	learn: 0.2406694	total: 10.5s	remaining: 20.4s
340:	learn: 0.2405199	total: 10.5s	remaining: 20.4s
341:	learn: 0.2402971	total: 10.6s	remaining: 20.3s
342:	learn: 0.2401651	total: 10.6s	remaining: 20.3s
343:	learn: 0.2399707	total: 10.6s	remaining: 20.3s
344:	learn: 0.2399703	total: 10.7s	remaining: 20.2s
345:	learn: 0.2397709	total: 10.7s	remaining: 20.2s
346:	learn: 0.2395274	total: 10.7s	remaining: 20.2s
347:	learn: 

487:	learn: 0.2199630	total: 14.9s	remaining: 15.7s
488:	learn: 0.2199238	total: 15s	remaining: 15.6s
489:	learn: 0.2197809	total: 15s	remaining: 15.6s
490:	learn: 0.2196767	total: 15s	remaining: 15.6s
491:	learn: 0.2195508	total: 15s	remaining: 15.5s
492:	learn: 0.2194879	total: 15.1s	remaining: 15.5s
493:	learn: 0.2193348	total: 15.1s	remaining: 15.5s
494:	learn: 0.2191752	total: 15.1s	remaining: 15.4s
495:	learn: 0.2189835	total: 15.2s	remaining: 15.4s
496:	learn: 0.2188084	total: 15.2s	remaining: 15.4s
497:	learn: 0.2186850	total: 15.2s	remaining: 15.4s
498:	learn: 0.2185216	total: 15.3s	remaining: 15.3s
499:	learn: 0.2183181	total: 15.3s	remaining: 15.3s
500:	learn: 0.2181837	total: 15.3s	remaining: 15.3s
501:	learn: 0.2180497	total: 15.4s	remaining: 15.2s
502:	learn: 0.2178970	total: 15.4s	remaining: 15.2s
503:	learn: 0.2176262	total: 15.4s	remaining: 15.2s
504:	learn: 0.2174447	total: 15.5s	remaining: 15.2s
505:	learn: 0.2172791	total: 15.5s	remaining: 15.1s
506:	learn: 0.217107

...

489:	learn: 0.0028677	total: 12.6s	remaining: 13.1s
490:	learn: 0.0028599	total: 12.6s	remaining: 13.1s
491:	learn: 0.0028554	total: 12.7s	remaining: 13.1s
492:	learn: 0.0028472	total: 12.7s	remaining: 13s
493:	learn: 0.0028436	total: 12.7s	remaining: 13s
494:	learn: 0.0028396	total: 12.7s	remaining: 13s
495:	learn: 0.0028358	total: 12.7s	remaining: 12.9s
496:	learn: 0.0028322	total: 12.8s	remaining: 12.9s
497:	learn: 0.0028297	total: 12.8s	remaining: 12.9s
498:	learn: 0.0028270	total: 12.8s	remaining: 12.9s
499:	learn: 0.0028195	total: 12.8s	remaining: 12.8s
500:	learn: 0.0028159	total: 12.8s	remaining: 12.8s
501:	learn: 0.0028121	total: 12.9s	remaining: 12.8s
502:	learn: 0.0028079	total: 12.9s	remaining: 12.7s
503:	learn: 0.0028021	total: 12.9s	remaining: 12.7s
504:	learn: 0.0027941	total: 12.9s	remaining: 12.7s
505:	learn: 0.0027925	total: 13s	remaining: 12.7s
506:	learn: 0.0027900	total: 13s	remaining: 12.6s
507:	learn: 0.0027865	total: 13s	remaining: 12.6s
508:	learn: 0.0027840	to

654:	learn: 0.0020211	total: 16.5s	remaining: 8.68s
655:	learn: 0.0020180	total: 16.5s	remaining: 8.65s
656:	learn: 0.0020143	total: 16.5s	remaining: 8.62s
657:	learn: 0.0020135	total: 16.5s	remaining: 8.6s
658:	learn: 0.0020112	total: 16.6s	remaining: 8.57s
659:	learn: 0.0020053	total: 16.6s	remaining: 8.54s
660:	learn: 0.0020037	total: 16.6s	remaining: 8.52s
661:	learn: 0.0020012	total: 16.6s	remaining: 8.49s
662:	learn: 0.0019997	total: 16.7s	remaining: 8.46s
663:	learn: 0.0019966	total: 16.7s	remaining: 8.44s
664:	learn: 0.0019958	total: 16.7s	remaining: 8.41s
665:	learn: 0.0019903	total: 16.7s	remaining: 8.39s
666:	learn: 0.0019831	total: 16.7s	remaining: 8.36s
667:	learn: 0.0019811	total: 16.8s	remaining: 8.33s
668:	learn: 0.0019751	total: 16.8s	remaining: 8.31s
669:	learn: 0.0019698	total: 16.8s	remaining: 8.28s
670:	learn: 0.0019653	total: 16.8s	remaining: 8.25s
671:	learn: 0.0019624	total: 16.9s	remaining: 8.23s
672:	learn: 0.0019606	total: 16.9s	remaining: 8.2s
673:	learn: 0.

822:	learn: 0.0015015	total: 20.3s	remaining: 4.37s
823:	learn: 0.0015006	total: 20.3s	remaining: 4.34s
824:	learn: 0.0014993	total: 20.4s	remaining: 4.32s
825:	learn: 0.0014975	total: 20.4s	remaining: 4.29s
826:	learn: 0.0014963	total: 20.4s	remaining: 4.27s
827:	learn: 0.0014950	total: 20.4s	remaining: 4.24s
828:	learn: 0.0014898	total: 20.5s	remaining: 4.22s
829:	learn: 0.0014873	total: 20.5s	remaining: 4.19s
830:	learn: 0.0014866	total: 20.5s	remaining: 4.17s
831:	learn: 0.0014831	total: 20.5s	remaining: 4.14s
832:	learn: 0.0014815	total: 20.5s	remaining: 4.12s
833:	learn: 0.0014810	total: 20.6s	remaining: 4.09s
834:	learn: 0.0014800	total: 20.6s	remaining: 4.07s
835:	learn: 0.0014780	total: 20.6s	remaining: 4.04s
836:	learn: 0.0014774	total: 20.6s	remaining: 4.02s
837:	learn: 0.0014767	total: 20.6s	remaining: 3.99s
838:	learn: 0.0014749	total: 20.7s	remaining: 3.97s
839:	learn: 0.0014739	total: 20.7s	remaining: 3.94s
840:	learn: 0.0014723	total: 20.7s	remaining: 3.92s
841:	learn: 

982:	learn: 0.0012081	total: 25s	remaining: 433ms
983:	learn: 0.0012057	total: 25.1s	remaining: 408ms
984:	learn: 0.0012053	total: 25.2s	remaining: 383ms
985:	learn: 0.0012028	total: 25.2s	remaining: 358ms
986:	learn: 0.0012025	total: 25.2s	remaining: 332ms
987:	learn: 0.0012025	total: 25.3s	remaining: 307ms
988:	learn: 0.0012021	total: 25.3s	remaining: 281ms
989:	learn: 0.0012016	total: 25.3s	remaining: 256ms
990:	learn: 0.0012008	total: 25.3s	remaining: 230ms
991:	learn: 0.0012002	total: 25.4s	remaining: 205ms
992:	learn: 0.0011990	total: 25.4s	remaining: 179ms
993:	learn: 0.0011985	total: 25.5s	remaining: 154ms
994:	learn: 0.0011946	total: 25.5s	remaining: 128ms
995:	learn: 0.0011933	total: 25.5s	remaining: 103ms
996:	learn: 0.0011915	total: 25.6s	remaining: 77ms
997:	learn: 0.0011907	total: 25.6s	remaining: 51.3ms
998:	learn: 0.0011877	total: 25.6s	remaining: 25.7ms
999:	learn: 0.0011842	total: 25.7s	remaining: 0us


VotingClassifier(estimators=[('lgb', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.05, max_depth=4, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_iterations=150, num_leaves=31, objective=None,...ty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft',
         weights=[1, 3, 1, 1])

In [58]:
base_model_evaluate(model_A, x_train_new_A, y_train_A)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6732111	total: 52.3ms	remaining: 52.3s
1:	learn: 0.6558177	total: 104ms	remaining: 51.7s
2:	learn: 0.6389678	total: 131ms	remaining: 43.7s
3:	learn: 0.6241370	total: 157ms	remaining: 39.1s
4:	learn: 0.6104116	total: 186ms	remaining: 37s
5:	learn: 0.5961376	total: 210ms	remaining: 34.8s
6:	learn: 0.5842138	total: 237ms	remaining: 33.7s
7:	learn: 0.5728070	total: 275ms	remaining: 34s
8:	learn: 0.5615229	total: 303ms	remaining: 33.4s
9:	learn: 0.5519561	total: 331ms	remaining: 32.8s
10:	learn: 0.5429668	total: 359ms	remaining: 32.2s
11:	learn: 0.5341470	total: 387ms	remaining: 31.9s
12:	learn: 0.5258953	total: 416ms	remaining: 31.6s
13:	learn: 0.5180908	total: 440ms	remaining: 31s
14:	learn: 0.5109267	total: 470ms	remaining: 30.9s
15:	learn: 0.5047072	total: 503ms	remaining: 31s
16:	learn: 0.4978323	total: 531ms	remaining: 30.7s
17:	learn: 0.4912981	total: 556ms	remaining: 30.3s
18:	learn: 0.4848530	total: 589ms	remaining: 30.4s
19:	learn: 0.4797607	total: 616ms	remaining: 30.

162:	learn: 0.2850851	total: 4.99s	remaining: 25.6s
163:	learn: 0.2847736	total: 5.01s	remaining: 25.6s
164:	learn: 0.2843886	total: 5.04s	remaining: 25.5s
165:	learn: 0.2839540	total: 5.07s	remaining: 25.5s
166:	learn: 0.2834473	total: 5.09s	remaining: 25.4s
167:	learn: 0.2830763	total: 5.12s	remaining: 25.4s
168:	learn: 0.2828191	total: 5.15s	remaining: 25.3s
169:	learn: 0.2824971	total: 5.17s	remaining: 25.3s
170:	learn: 0.2820379	total: 5.2s	remaining: 25.2s
171:	learn: 0.2814932	total: 5.23s	remaining: 25.2s
172:	learn: 0.2810544	total: 5.26s	remaining: 25.1s
173:	learn: 0.2806547	total: 5.28s	remaining: 25.1s
174:	learn: 0.2803770	total: 5.31s	remaining: 25.1s
175:	learn: 0.2800057	total: 5.34s	remaining: 25s
176:	learn: 0.2795606	total: 5.37s	remaining: 25s
177:	learn: 0.2793428	total: 5.4s	remaining: 24.9s
178:	learn: 0.2789364	total: 5.42s	remaining: 24.9s
179:	learn: 0.2785887	total: 5.45s	remaining: 24.8s
180:	learn: 0.2779901	total: 5.48s	remaining: 24.8s
181:	learn: 0.2774

321:	learn: 0.2386009	total: 9.34s	remaining: 19.7s
322:	learn: 0.2383835	total: 9.37s	remaining: 19.6s
323:	learn: 0.2381673	total: 9.39s	remaining: 19.6s
324:	learn: 0.2378029	total: 9.42s	remaining: 19.6s
325:	learn: 0.2375738	total: 9.45s	remaining: 19.5s
326:	learn: 0.2372488	total: 9.47s	remaining: 19.5s
327:	learn: 0.2370365	total: 9.5s	remaining: 19.5s
328:	learn: 0.2368035	total: 9.53s	remaining: 19.4s
329:	learn: 0.2365724	total: 9.56s	remaining: 19.4s
330:	learn: 0.2364011	total: 9.58s	remaining: 19.4s
331:	learn: 0.2362163	total: 9.61s	remaining: 19.3s
332:	learn: 0.2359714	total: 9.64s	remaining: 19.3s
333:	learn: 0.2357681	total: 9.67s	remaining: 19.3s
334:	learn: 0.2355566	total: 9.71s	remaining: 19.3s
335:	learn: 0.2353770	total: 9.73s	remaining: 19.2s
336:	learn: 0.2351835	total: 9.76s	remaining: 19.2s
337:	learn: 0.2349615	total: 9.79s	remaining: 19.2s
338:	learn: 0.2347377	total: 9.81s	remaining: 19.1s
339:	learn: 0.2345702	total: 9.84s	remaining: 19.1s
340:	learn: 0

481:	learn: 0.2064393	total: 13.7s	remaining: 14.7s
482:	learn: 0.2063153	total: 13.7s	remaining: 14.7s
483:	learn: 0.2061706	total: 13.8s	remaining: 14.7s
484:	learn: 0.2059003	total: 13.8s	remaining: 14.6s
485:	learn: 0.2057467	total: 13.8s	remaining: 14.6s
486:	learn: 0.2055533	total: 13.8s	remaining: 14.6s
487:	learn: 0.2054413	total: 13.9s	remaining: 14.6s
488:	learn: 0.2053210	total: 13.9s	remaining: 14.5s
489:	learn: 0.2051359	total: 13.9s	remaining: 14.5s
490:	learn: 0.2049514	total: 14s	remaining: 14.5s
491:	learn: 0.2048088	total: 14s	remaining: 14.4s
492:	learn: 0.2046545	total: 14s	remaining: 14.4s
493:	learn: 0.2045423	total: 14s	remaining: 14.4s
494:	learn: 0.2044216	total: 14.1s	remaining: 14.4s
495:	learn: 0.2042658	total: 14.1s	remaining: 14.3s
496:	learn: 0.2041591	total: 14.1s	remaining: 14.3s
497:	learn: 0.2039800	total: 14.2s	remaining: 14.3s
498:	learn: 0.2038094	total: 14.2s	remaining: 14.2s
499:	learn: 0.2036569	total: 14.2s	remaining: 14.2s
500:	learn: 0.203551

...

486:	learn: 0.2088452	total: 14.5s	remaining: 15.3s
487:	learn: 0.2087039	total: 14.6s	remaining: 15.3s
488:	learn: 0.2085427	total: 14.6s	remaining: 15.3s
489:	learn: 0.2082980	total: 14.6s	remaining: 15.2s
490:	learn: 0.2081012	total: 14.7s	remaining: 15.2s
491:	learn: 0.2078686	total: 14.7s	remaining: 15.2s
492:	learn: 0.2078686	total: 14.7s	remaining: 15.1s
493:	learn: 0.2078684	total: 14.7s	remaining: 15.1s
494:	learn: 0.2077000	total: 14.8s	remaining: 15.1s
495:	learn: 0.2075451	total: 14.8s	remaining: 15s
496:	learn: 0.2075450	total: 14.8s	remaining: 15s
497:	learn: 0.2072865	total: 14.8s	remaining: 14.9s
498:	learn: 0.2070430	total: 14.9s	remaining: 14.9s
499:	learn: 0.2067819	total: 14.9s	remaining: 14.9s
500:	learn: 0.2067817	total: 14.9s	remaining: 14.8s
501:	learn: 0.2065455	total: 14.9s	remaining: 14.8s
502:	learn: 0.2063805	total: 15s	remaining: 14.8s
503:	learn: 0.2061278	total: 15s	remaining: 14.7s
504:	learn: 0.2059153	total: 15s	remaining: 14.7s
505:	learn: 0.2057139	

651:	learn: 0.1879521	total: 18.8s	remaining: 10s
652:	learn: 0.1877793	total: 18.8s	remaining: 10s
653:	learn: 0.1876607	total: 18.9s	remaining: 9.98s
654:	learn: 0.1874523	total: 18.9s	remaining: 9.95s
655:	learn: 0.1874271	total: 18.9s	remaining: 9.92s
656:	learn: 0.1873499	total: 18.9s	remaining: 9.89s
657:	learn: 0.1871949	total: 19s	remaining: 9.86s
658:	learn: 0.1870616	total: 19s	remaining: 9.83s
659:	learn: 0.1869074	total: 19s	remaining: 9.8s
660:	learn: 0.1867855	total: 19.1s	remaining: 9.77s
661:	learn: 0.1866438	total: 19.1s	remaining: 9.74s
662:	learn: 0.1864516	total: 19.1s	remaining: 9.71s
663:	learn: 0.1863493	total: 19.1s	remaining: 9.69s
664:	learn: 0.1861462	total: 19.2s	remaining: 9.65s
665:	learn: 0.1859744	total: 19.2s	remaining: 9.63s
666:	learn: 0.1859059	total: 19.2s	remaining: 9.6s
667:	learn: 0.1858875	total: 19.3s	remaining: 9.57s
668:	learn: 0.1857246	total: 19.3s	remaining: 9.54s
669:	learn: 0.1855833	total: 19.3s	remaining: 9.51s
670:	learn: 0.1854197	to

818:	learn: 0.1718621	total: 23.1s	remaining: 5.12s
819:	learn: 0.1717490	total: 23.2s	remaining: 5.09s
820:	learn: 0.1716168	total: 23.2s	remaining: 5.06s
821:	learn: 0.1715246	total: 23.2s	remaining: 5.03s
822:	learn: 0.1715213	total: 23.3s	remaining: 5s
823:	learn: 0.1714368	total: 23.3s	remaining: 4.97s
824:	learn: 0.1713473	total: 23.3s	remaining: 4.95s
825:	learn: 0.1712449	total: 23.3s	remaining: 4.92s
826:	learn: 0.1712403	total: 23.4s	remaining: 4.89s
827:	learn: 0.1712402	total: 23.4s	remaining: 4.86s
828:	learn: 0.1710594	total: 23.4s	remaining: 4.83s
829:	learn: 0.1708402	total: 23.4s	remaining: 4.8s
830:	learn: 0.1706970	total: 23.5s	remaining: 4.77s
831:	learn: 0.1706970	total: 23.5s	remaining: 4.74s
832:	learn: 0.1705717	total: 23.5s	remaining: 4.71s
833:	learn: 0.1704575	total: 23.5s	remaining: 4.69s
834:	learn: 0.1702520	total: 23.6s	remaining: 4.66s
835:	learn: 0.1700690	total: 23.6s	remaining: 4.63s
836:	learn: 0.1699349	total: 23.6s	remaining: 4.6s
837:	learn: 0.169

982:	learn: 0.1552693	total: 27.7s	remaining: 479ms
983:	learn: 0.1551230	total: 27.7s	remaining: 451ms
984:	learn: 0.1550005	total: 27.8s	remaining: 423ms
985:	learn: 0.1548845	total: 27.8s	remaining: 395ms
986:	learn: 0.1547034	total: 27.8s	remaining: 366ms
987:	learn: 0.1546021	total: 27.9s	remaining: 338ms
988:	learn: 0.1545163	total: 27.9s	remaining: 310ms
989:	learn: 0.1545163	total: 27.9s	remaining: 282ms
990:	learn: 0.1545153	total: 27.9s	remaining: 254ms
991:	learn: 0.1543461	total: 28s	remaining: 225ms
992:	learn: 0.1542385	total: 28s	remaining: 197ms
993:	learn: 0.1541264	total: 28s	remaining: 169ms
994:	learn: 0.1540493	total: 28s	remaining: 141ms
995:	learn: 0.1538721	total: 28.1s	remaining: 113ms
996:	learn: 0.1537643	total: 28.1s	remaining: 84.5ms
997:	learn: 0.1536932	total: 28.1s	remaining: 56.3ms
998:	learn: 0.1535464	total: 28.1s	remaining: 28.2ms
999:	learn: 0.1534792	total: 28.2s	remaining: 0us


(0.2664197102812849, 0.9557871300450248)

In [59]:
base_model_evaluate(model_B, x_train_new_B, y_train_B)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6603770	total: 47.9ms	remaining: 47.8s
1:	learn: 0.6277601	total: 101ms	remaining: 50.4s
2:	learn: 0.5944975	total: 132ms	remaining: 44s
3:	learn: 0.5678994	total: 152ms	remaining: 37.9s
4:	learn: 0.5406757	total: 185ms	remaining: 36.8s
5:	learn: 0.5184088	total: 205ms	remaining: 33.9s
6:	learn: 0.4972315	total: 233ms	remaining: 33s
7:	learn: 0.4762056	total: 257ms	remaining: 31.9s
8:	learn: 0.4574685	total: 283ms	remaining: 31.1s
9:	learn: 0.4404930	total: 302ms	remaining: 29.9s
10:	learn: 0.4245585	total: 330ms	remaining: 29.7s
11:	learn: 0.4086630	total: 349ms	remaining: 28.7s
12:	learn: 0.3950291	total: 380ms	remaining: 28.9s
13:	learn: 0.3833811	total: 401ms	remaining: 28.2s
14:	learn: 0.3709784	total: 430ms	remaining: 28.2s
15:	learn: 0.3611148	total: 450ms	remaining: 27.7s
16:	learn: 0.3512318	total: 480ms	remaining: 27.7s
17:	learn: 0.3420265	total: 498ms	remaining: 27.2s
18:	learn: 0.3334657	total: 520ms	remaining: 26.8s
19:	learn: 0.3263743	total: 535ms	remaining:

164:	learn: 0.1406087	total: 3.73s	remaining: 18.9s
165:	learn: 0.1402907	total: 3.76s	remaining: 18.9s
166:	learn: 0.1398280	total: 3.78s	remaining: 18.8s
167:	learn: 0.1392516	total: 3.8s	remaining: 18.8s
168:	learn: 0.1389012	total: 3.82s	remaining: 18.8s
169:	learn: 0.1383581	total: 3.84s	remaining: 18.7s
170:	learn: 0.1378168	total: 3.86s	remaining: 18.7s
171:	learn: 0.1374875	total: 3.88s	remaining: 18.7s
172:	learn: 0.1371771	total: 3.9s	remaining: 18.6s
173:	learn: 0.1366417	total: 3.92s	remaining: 18.6s
174:	learn: 0.1363363	total: 3.94s	remaining: 18.6s
175:	learn: 0.1361039	total: 3.96s	remaining: 18.5s
176:	learn: 0.1356998	total: 3.98s	remaining: 18.5s
177:	learn: 0.1354049	total: 4s	remaining: 18.5s
178:	learn: 0.1350564	total: 4.02s	remaining: 18.4s
179:	learn: 0.1346184	total: 4.04s	remaining: 18.4s
180:	learn: 0.1342447	total: 4.06s	remaining: 18.4s
181:	learn: 0.1337564	total: 4.08s	remaining: 18.4s
182:	learn: 0.1332780	total: 4.11s	remaining: 18.4s
183:	learn: 0.132

332:	learn: 0.0903952	total: 7.56s	remaining: 15.1s
333:	learn: 0.0900280	total: 7.58s	remaining: 15.1s
334:	learn: 0.0897603	total: 7.61s	remaining: 15.1s
335:	learn: 0.0894856	total: 7.63s	remaining: 15.1s
336:	learn: 0.0893179	total: 7.65s	remaining: 15.1s
337:	learn: 0.0889966	total: 7.67s	remaining: 15s
338:	learn: 0.0888625	total: 7.7s	remaining: 15s
339:	learn: 0.0887537	total: 7.72s	remaining: 15s
340:	learn: 0.0885362	total: 7.74s	remaining: 15s
341:	learn: 0.0881437	total: 7.76s	remaining: 14.9s
342:	learn: 0.0877552	total: 7.78s	remaining: 14.9s
343:	learn: 0.0876064	total: 7.8s	remaining: 14.9s
344:	learn: 0.0874019	total: 7.82s	remaining: 14.8s
345:	learn: 0.0872618	total: 7.84s	remaining: 14.8s
346:	learn: 0.0870128	total: 7.86s	remaining: 14.8s
347:	learn: 0.0866692	total: 7.88s	remaining: 14.8s
348:	learn: 0.0864932	total: 7.9s	remaining: 14.7s
349:	learn: 0.0862451	total: 7.92s	remaining: 14.7s
350:	learn: 0.0860545	total: 7.94s	remaining: 14.7s
351:	learn: 0.0859454	t

492:	learn: 0.0613464	total: 11s	remaining: 11.3s
493:	learn: 0.0612236	total: 11s	remaining: 11.3s
494:	learn: 0.0610234	total: 11s	remaining: 11.2s
495:	learn: 0.0608264	total: 11s	remaining: 11.2s
496:	learn: 0.0607399	total: 11.1s	remaining: 11.2s
497:	learn: 0.0606359	total: 11.1s	remaining: 11.2s
498:	learn: 0.0605151	total: 11.1s	remaining: 11.1s
499:	learn: 0.0603554	total: 11.1s	remaining: 11.1s
500:	learn: 0.0601933	total: 11.1s	remaining: 11.1s
501:	learn: 0.0600785	total: 11.2s	remaining: 11.1s
502:	learn: 0.0599179	total: 11.2s	remaining: 11.1s
503:	learn: 0.0598221	total: 11.2s	remaining: 11s
504:	learn: 0.0596196	total: 11.2s	remaining: 11s
505:	learn: 0.0595285	total: 11.3s	remaining: 11s
506:	learn: 0.0593674	total: 11.3s	remaining: 11s
507:	learn: 0.0591850	total: 11.3s	remaining: 10.9s
508:	learn: 0.0588189	total: 11.3s	remaining: 10.9s
509:	learn: 0.0587035	total: 11.3s	remaining: 10.9s
510:	learn: 0.0585370	total: 11.4s	remaining: 10.9s
511:	learn: 0.0583650	total:

...

490:	learn: 0.0641406	total: 16.6s	remaining: 17.2s
491:	learn: 0.0639087	total: 16.6s	remaining: 17.1s
492:	learn: 0.0637250	total: 16.6s	remaining: 17.1s
493:	learn: 0.0635331	total: 16.6s	remaining: 17s
494:	learn: 0.0632612	total: 16.7s	remaining: 17s
495:	learn: 0.0631689	total: 16.7s	remaining: 17s
496:	learn: 0.0629826	total: 16.7s	remaining: 16.9s
497:	learn: 0.0628338	total: 16.8s	remaining: 16.9s
498:	learn: 0.0626525	total: 16.8s	remaining: 16.9s
499:	learn: 0.0624887	total: 16.8s	remaining: 16.8s
500:	learn: 0.0623181	total: 16.8s	remaining: 16.8s
501:	learn: 0.0621867	total: 16.9s	remaining: 16.7s
502:	learn: 0.0620748	total: 16.9s	remaining: 16.7s
503:	learn: 0.0618731	total: 16.9s	remaining: 16.7s
504:	learn: 0.0617555	total: 17s	remaining: 16.6s
505:	learn: 0.0615891	total: 17s	remaining: 16.6s
506:	learn: 0.0615156	total: 17s	remaining: 16.5s
507:	learn: 0.0613803	total: 17s	remaining: 16.5s
508:	learn: 0.0611753	total: 17.1s	remaining: 16.5s
509:	learn: 0.0610849	tota

657:	learn: 0.0430703	total: 21.6s	remaining: 11.2s
658:	learn: 0.0429901	total: 21.6s	remaining: 11.2s
659:	learn: 0.0429274	total: 21.7s	remaining: 11.2s
660:	learn: 0.0428811	total: 21.7s	remaining: 11.1s
661:	learn: 0.0427356	total: 21.7s	remaining: 11.1s
662:	learn: 0.0426269	total: 21.8s	remaining: 11.1s
663:	learn: 0.0425649	total: 21.8s	remaining: 11s
664:	learn: 0.0424773	total: 21.8s	remaining: 11s
665:	learn: 0.0423856	total: 21.9s	remaining: 11s
666:	learn: 0.0421843	total: 21.9s	remaining: 10.9s
667:	learn: 0.0420904	total: 21.9s	remaining: 10.9s
668:	learn: 0.0419708	total: 21.9s	remaining: 10.9s
669:	learn: 0.0419047	total: 22s	remaining: 10.8s
670:	learn: 0.0417878	total: 22s	remaining: 10.8s
671:	learn: 0.0417117	total: 22s	remaining: 10.8s
672:	learn: 0.0416321	total: 22.1s	remaining: 10.7s
673:	learn: 0.0415745	total: 22.1s	remaining: 10.7s
674:	learn: 0.0414522	total: 22.1s	remaining: 10.7s
675:	learn: 0.0412875	total: 22.2s	remaining: 10.6s
676:	learn: 0.0412338	to

822:	learn: 0.0306954	total: 26.8s	remaining: 5.77s
823:	learn: 0.0306606	total: 26.8s	remaining: 5.73s
824:	learn: 0.0306329	total: 26.9s	remaining: 5.7s
825:	learn: 0.0305889	total: 26.9s	remaining: 5.67s
826:	learn: 0.0305046	total: 26.9s	remaining: 5.63s
827:	learn: 0.0304522	total: 27s	remaining: 5.6s
828:	learn: 0.0304058	total: 27s	remaining: 5.57s
829:	learn: 0.0303052	total: 27s	remaining: 5.53s
830:	learn: 0.0302652	total: 27s	remaining: 5.5s
831:	learn: 0.0302419	total: 27.1s	remaining: 5.46s
832:	learn: 0.0301124	total: 27.1s	remaining: 5.43s
833:	learn: 0.0300742	total: 27.1s	remaining: 5.4s
834:	learn: 0.0300332	total: 27.1s	remaining: 5.36s
835:	learn: 0.0299815	total: 27.2s	remaining: 5.33s
836:	learn: 0.0299628	total: 27.2s	remaining: 5.3s
837:	learn: 0.0299393	total: 27.2s	remaining: 5.26s
838:	learn: 0.0298422	total: 27.3s	remaining: 5.23s
839:	learn: 0.0297125	total: 27.3s	remaining: 5.2s
840:	learn: 0.0296913	total: 27.3s	remaining: 5.16s
841:	learn: 0.0296267	tota

987:	learn: 0.0222303	total: 31.7s	remaining: 385ms
988:	learn: 0.0221714	total: 31.7s	remaining: 353ms
989:	learn: 0.0221358	total: 31.7s	remaining: 321ms
990:	learn: 0.0220843	total: 31.8s	remaining: 288ms
991:	learn: 0.0220107	total: 31.8s	remaining: 256ms
992:	learn: 0.0219514	total: 31.8s	remaining: 224ms
993:	learn: 0.0219272	total: 31.9s	remaining: 192ms
994:	learn: 0.0218626	total: 31.9s	remaining: 160ms
995:	learn: 0.0218170	total: 31.9s	remaining: 128ms
996:	learn: 0.0217933	total: 31.9s	remaining: 96.1ms
997:	learn: 0.0217651	total: 32s	remaining: 64.1ms
998:	learn: 0.0217408	total: 32s	remaining: 32ms
999:	learn: 0.0217175	total: 32s	remaining: 0us


(0.19458254413169676, 0.8833414100681871)

In [60]:
base_model_evaluate(model_C, x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6241760	total: 14.8ms	remaining: 14.8s
1:	learn: 0.5603592	total: 29.6ms	remaining: 14.8s
2:	learn: 0.5057866	total: 45ms	remaining: 14.9s
3:	learn: 0.4555718	total: 62ms	remaining: 15.4s
4:	learn: 0.4146371	total: 82.4ms	remaining: 16.4s
5:	learn: 0.3731055	total: 101ms	remaining: 16.7s
6:	learn: 0.3294482	total: 116ms	remaining: 16.5s
7:	learn: 0.2988070	total: 132ms	remaining: 16.4s
8:	learn: 0.2656629	total: 149ms	remaining: 16.4s
9:	learn: 0.2420742	total: 164ms	remaining: 16.3s
10:	learn: 0.2212773	total: 180ms	remaining: 16.1s
11:	learn: 0.2053032	total: 219ms	remaining: 18.1s
12:	learn: 0.1909827	total: 296ms	remaining: 22.5s
13:	learn: 0.1769203	total: 337ms	remaining: 23.7s
14:	learn: 0.1621377	total: 355ms	remaining: 23.3s
15:	learn: 0.1495952	total: 373ms	remaining: 22.9s
16:	learn: 0.1389676	total: 389ms	remaining: 22.5s
17:	learn: 0.1300735	total: 407ms	remaining: 22.2s
18:	learn: 0.1214134	total: 436ms	remaining: 22.5s
19:	learn: 0.1143063	total: 473ms	remain

169:	learn: 0.0091191	total: 4.49s	remaining: 21.9s
170:	learn: 0.0090761	total: 4.51s	remaining: 21.9s
171:	learn: 0.0089844	total: 4.54s	remaining: 21.9s
172:	learn: 0.0089478	total: 4.57s	remaining: 21.8s
173:	learn: 0.0088789	total: 4.59s	remaining: 21.8s
174:	learn: 0.0088015	total: 4.61s	remaining: 21.8s
175:	learn: 0.0087418	total: 4.64s	remaining: 21.7s
176:	learn: 0.0087038	total: 4.66s	remaining: 21.7s
177:	learn: 0.0086468	total: 4.68s	remaining: 21.6s
178:	learn: 0.0086092	total: 4.75s	remaining: 21.8s
179:	learn: 0.0085550	total: 4.8s	remaining: 21.9s
180:	learn: 0.0085005	total: 4.82s	remaining: 21.8s
181:	learn: 0.0084643	total: 4.84s	remaining: 21.7s
182:	learn: 0.0084156	total: 4.85s	remaining: 21.7s
183:	learn: 0.0083533	total: 4.87s	remaining: 21.6s
184:	learn: 0.0083250	total: 4.88s	remaining: 21.5s
185:	learn: 0.0082706	total: 4.9s	remaining: 21.4s
186:	learn: 0.0082313	total: 4.92s	remaining: 21.4s
187:	learn: 0.0081879	total: 4.93s	remaining: 21.3s
188:	learn: 0.

337:	learn: 0.0039121	total: 7.71s	remaining: 15.1s
338:	learn: 0.0038896	total: 7.74s	remaining: 15.1s
339:	learn: 0.0038748	total: 7.76s	remaining: 15.1s
340:	learn: 0.0038599	total: 7.78s	remaining: 15s
341:	learn: 0.0038285	total: 7.8s	remaining: 15s
342:	learn: 0.0038171	total: 7.81s	remaining: 15s
343:	learn: 0.0038019	total: 7.83s	remaining: 14.9s
344:	learn: 0.0037642	total: 7.85s	remaining: 14.9s
345:	learn: 0.0037569	total: 7.86s	remaining: 14.9s
346:	learn: 0.0037468	total: 7.88s	remaining: 14.8s
347:	learn: 0.0037368	total: 7.9s	remaining: 14.8s
348:	learn: 0.0037178	total: 7.97s	remaining: 14.9s
349:	learn: 0.0036969	total: 8s	remaining: 14.9s
350:	learn: 0.0036881	total: 8.04s	remaining: 14.9s
351:	learn: 0.0036662	total: 8.11s	remaining: 14.9s
352:	learn: 0.0036461	total: 8.18s	remaining: 15s
353:	learn: 0.0036349	total: 8.22s	remaining: 15s
354:	learn: 0.0036251	total: 8.26s	remaining: 15s
355:	learn: 0.0036111	total: 8.28s	remaining: 15s
356:	learn: 0.0035957	total: 8.

498:	learn: 0.0021198	total: 11.7s	remaining: 11.7s
499:	learn: 0.0021098	total: 11.7s	remaining: 11.7s
500:	learn: 0.0021073	total: 11.7s	remaining: 11.6s
501:	learn: 0.0021020	total: 11.7s	remaining: 11.6s
502:	learn: 0.0020949	total: 11.7s	remaining: 11.6s
503:	learn: 0.0020924	total: 11.7s	remaining: 11.6s
504:	learn: 0.0020869	total: 11.8s	remaining: 11.5s
505:	learn: 0.0020755	total: 11.8s	remaining: 11.5s
506:	learn: 0.0020603	total: 11.8s	remaining: 11.5s
507:	learn: 0.0020549	total: 11.8s	remaining: 11.4s
508:	learn: 0.0020457	total: 11.8s	remaining: 11.4s
509:	learn: 0.0020346	total: 11.8s	remaining: 11.4s
510:	learn: 0.0020288	total: 11.9s	remaining: 11.3s
511:	learn: 0.0020181	total: 11.9s	remaining: 11.3s
512:	learn: 0.0020080	total: 11.9s	remaining: 11.3s
513:	learn: 0.0019981	total: 11.9s	remaining: 11.3s
514:	learn: 0.0019949	total: 11.9s	remaining: 11.2s
515:	learn: 0.0019834	total: 11.9s	remaining: 11.2s
516:	learn: 0.0019704	total: 12s	remaining: 11.2s
517:	learn: 0.

...

653:	learn: 0.0012151	total: 15.1s	remaining: 7.97s
654:	learn: 0.0012080	total: 15.1s	remaining: 7.96s
655:	learn: 0.0012065	total: 15.2s	remaining: 7.95s
656:	learn: 0.0012037	total: 15.2s	remaining: 7.94s
657:	learn: 0.0012014	total: 15.3s	remaining: 7.96s
658:	learn: 0.0011994	total: 15.4s	remaining: 7.97s
659:	learn: 0.0011967	total: 15.5s	remaining: 7.98s
660:	learn: 0.0011933	total: 15.6s	remaining: 8s
661:	learn: 0.0011848	total: 15.7s	remaining: 8s
662:	learn: 0.0011829	total: 15.7s	remaining: 7.99s
663:	learn: 0.0011816	total: 15.9s	remaining: 8.03s
664:	learn: 0.0011774	total: 16s	remaining: 8.06s
665:	learn: 0.0011747	total: 16.1s	remaining: 8.06s
666:	learn: 0.0011717	total: 16.2s	remaining: 8.07s
667:	learn: 0.0011706	total: 16.2s	remaining: 8.07s
668:	learn: 0.0011652	total: 16.3s	remaining: 8.06s
669:	learn: 0.0011613	total: 16.3s	remaining: 8.04s
670:	learn: 0.0011575	total: 16.4s	remaining: 8.03s
671:	learn: 0.0011493	total: 16.4s	remaining: 8.01s
672:	learn: 0.001148

816:	learn: 0.0008698	total: 21.8s	remaining: 4.89s
817:	learn: 0.0008687	total: 21.8s	remaining: 4.86s
818:	learn: 0.0008666	total: 21.9s	remaining: 4.83s
819:	learn: 0.0008659	total: 21.9s	remaining: 4.8s
820:	learn: 0.0008609	total: 21.9s	remaining: 4.77s
821:	learn: 0.0008593	total: 21.9s	remaining: 4.75s
822:	learn: 0.0008585	total: 21.9s	remaining: 4.72s
823:	learn: 0.0008579	total: 22s	remaining: 4.69s
824:	learn: 0.0008551	total: 22s	remaining: 4.66s
825:	learn: 0.0008538	total: 22s	remaining: 4.64s
826:	learn: 0.0008527	total: 22s	remaining: 4.61s
827:	learn: 0.0008520	total: 22.1s	remaining: 4.58s
828:	learn: 0.0008495	total: 22.1s	remaining: 4.55s
829:	learn: 0.0008465	total: 22.1s	remaining: 4.53s
830:	learn: 0.0008428	total: 22.1s	remaining: 4.5s
831:	learn: 0.0008418	total: 22.1s	remaining: 4.47s
832:	learn: 0.0008414	total: 22.2s	remaining: 4.44s
833:	learn: 0.0008390	total: 22.2s	remaining: 4.41s
834:	learn: 0.0008363	total: 22.2s	remaining: 4.38s
835:	learn: 0.0008345	

978:	learn: 0.0006752	total: 25s	remaining: 537ms
979:	learn: 0.0006749	total: 25s	remaining: 511ms
980:	learn: 0.0006743	total: 25.1s	remaining: 485ms
981:	learn: 0.0006708	total: 25.1s	remaining: 460ms
982:	learn: 0.0006695	total: 25.1s	remaining: 434ms
983:	learn: 0.0006677	total: 25.1s	remaining: 408ms
984:	learn: 0.0006660	total: 25.1s	remaining: 383ms
985:	learn: 0.0006654	total: 25.2s	remaining: 357ms
986:	learn: 0.0006651	total: 25.2s	remaining: 332ms
987:	learn: 0.0006638	total: 25.2s	remaining: 306ms
988:	learn: 0.0006613	total: 25.2s	remaining: 280ms
989:	learn: 0.0006607	total: 25.2s	remaining: 255ms
990:	learn: 0.0006588	total: 25.3s	remaining: 229ms
991:	learn: 0.0006585	total: 25.3s	remaining: 204ms
992:	learn: 0.0006557	total: 25.3s	remaining: 178ms
993:	learn: 0.0006548	total: 25.3s	remaining: 153ms
994:	learn: 0.0006541	total: 25.3s	remaining: 127ms
995:	learn: 0.0006537	total: 25.4s	remaining: 102ms
996:	learn: 0.0006533	total: 25.4s	remaining: 76.4ms
997:	learn: 0.0

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


(0.016204901209516325, 0.9995916892353165)

In [61]:
A_pred = model_A.predict_proba(x_test_new_A)
B_pred = model_B.predict_proba(x_test_new_B)
C_pred = model_C.predict_proba(x_test_new_C)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [62]:
A_sub = make_country_sub(A_pred, test_A, 'A')
B_sub = make_country_sub(B_pred, test_B, 'B')
C_sub = make_country_sub(C_pred, test_C, 'C')

In [63]:
submission = pd.concat([A_sub, B_sub, C_sub])

In [64]:
submission.to_csv('0225.csv')